In [1]:
from create_playlist import Playlist
import os
import random

In [2]:
"""
Playlist:

tree = XML Element Tree, contains Playlist Element. 
Playlist Element contains paths to video files, and thus is what is edited to structure playlists
Videos added to playlist XML via add_track(path_to_track) method call

add_track() - Adds path to track file to XML

get_playlist() - Returns playlist with tracks added
"""
class Playlist:
    """Build xml playlist."""

    def __init__(self):
        #Defines basic tree structure.
        self.playlist = xml.Element('playlist')
        self.tree = xml.ElementTree(self.playlist)
        self.playlist.set('xmlns','http://xspf.org/ns/0/')
        self.playlist.set('xmlns:vlc','http://www.videolan.org/vlc/playlist/ns/0/')
        self.playlist.set('version', '1')

        self.title = xml.Element('title')
        self.playlist.append(self.title)
        self.title.text = 'Playlist'

        self.trackList = xml.Element('trackList')
        self.playlist.append(self.trackList)

    def add_track(self, path):
        #Add tracks to xml tree (within trackList).
        track = xml.Element('track')
        location = xml.Element('location')
        location.text = path
        track.append(location)
        self.trackList.append(track)

    def get_playlist(self):
        #Return complete playlist with tracks.
        return self.playlist

In [3]:
"""
Videos:

Utility object for producing videos lists. Does not contain data.

remove_nonvideo_files() - Takes list of files and removes filenames with wrong extensions
edit_paths() - Takes list of video files and formats them with path and prefix as req'd by playlist file
get_videos() - Returns list of video files in directory
"""

"\nVideos:\n\nUtility object for producing videos lists. Does not contain data.\n\nremove_nonvideo_files() - Takes list of files and removes filenames with wrong extensions\nedit_paths() - Takes list of video files and formats them with path and prefix as req'd by playlist file\nget_videos() - Returns list of video files in directory\n"

In [4]:
"""
Library Usage Pattern:

Create Playlist and Videos objects. Get list of video files using get_videos() and format the paths using edit_paths().

Loop through paths and use add_track() to add the paths to each track to the playlist XML of the Playlist Object.

Would want to break files into series, or have subclass of Videos, called Series. Need some structure to be able to pull from 
in sequential order but from each stream randomly. Probably want to ignore the Videos Class, create my own XML producing Class
"""

'\nLibrary Usage Pattern:\n\nCreate Playlist and Videos objects. Get list of video files using get_videos() and format the paths using edit_paths().\n\nLoop through paths and use add_track() to add the paths to each track to the playlist XML of the Playlist Object.\n\nWould want to break files into series, or have subclass of Videos, called Series. Need some structure to be able to pull from \nin sequential order but from each stream randomly. Probably want to ignore the Videos Class, create my own XML producing Class\n'

In [5]:
class Series:
    """Manages the video files for a group of related sequential media files"""
    def __init__(self, series_dir=None):
        self.files = []
        if series_dir: 
            self.collect_files(series_dir)
        
    def remove_nonvideo_files(self, file_list):
    #Removes files whose extension is not mentioned in ext_list from list of files.
        ext_list = ['.mp4', '.mkv', '.avi', '.flv', '.mov', '.wmv', '.vob', '.mpg','.3gp', '.m4v']
        for index, file_name in enumerate(file_list[:]):
            if file_name.endswith(tuple(ext_list)) or file_name.endswith(tuple(ext.upper() for ext in ext_list)):
                pass
            else:
                file_list.remove(file_name)
        return file_list
    
    def get_edited_paths(self):
    #Add path and prefix to files as required in vlc playlist file. 
        video_files = self.files
        for index in range(len(video_files)):
            video_files[index] =( 
            'file:///' + os.path.join(video_files[index])).replace('\\','/')
        return video_files
    
    def collect_files(self, direc, check_subdirs = True):
        os.chdir(direc)
        if check_subdirs == True:
            pathlist = [os.getcwd()]
            for root, dirs, files in os.walk(os.getcwd()):
                for name in dirs:
                    subdir_path = os.path.join(root, name)
                    if subdir_path.find('\.') != -1:
                        pass
                    else:
                        pathlist.append(subdir_path)
            for path in pathlist:
                all_files = os.listdir(path)
                for f in self.remove_nonvideo_files(all_files):
                    location = path + '\\' + f
                    self.files.append(location)
        else:
            all_files = os.listdir(path)
            for f in self.remove_nonvideo_files(all_files):
                location = os.getcwd() + '\\' + f
                self.files.append(location)
        
                    

In [6]:
# Add Series name, directory association

In [7]:
class SeriesPlaylistRandomizer:
    """Class to randomly choose and produce an ordered Playlist
    from 1+ series in a random order, but sequential within each series"""
    def __init__(self):
        self.seriesPlaceDict = dict()
        self.seriesDict = dict()
        self.seriesLengthDict = dict()
    
    def add_series(self, seriesName:str, series:Series):
        self.seriesDict[seriesName] = series
        self.seriesPlaceDict[seriesName] = 0
        self.seriesLengthDict[seriesName] = len(series.files)
    
    def _increment_series_place(self, seriesName):
        self.seriesPlaceDict[seriesName] += 1
    
    def _get_next_track(self, seriesName):
        # Get filepath for next track in series according to the current series place
        return self.seriesDict[seriesName].files[self.seriesPlaceDict[seriesName]]
    
    def get_randomized_playlist(self):
        # Randomly choose from each series until seriesPlace = seriesLength
        playlist_videos_xml = []
        finished_series = []
        while finished_series != list(self.seriesDict.keys()):
            series = random.choice(list(self.seriesDict.keys()))
            if series in finished_series:
                continue
            else:
                next_track = self._get_next_track(series)
                edited_paths = next_track.get_edited_paths()
                playlist_videos_xml.append(edited_paths)
                self._increment_series_place(series)
                if self.seriesPlaceDict[series] == self.seriesLengthDict[series]:
                    finished_series.append(series)
        return playlist_videos_xml

In [8]:
series_1 = Series()
series_1.collect_files(r'C:\Users\Cameron Phillips\Downloads\Movies\[CBM] Cowboy Bebop 1-26 Complete (Dual Audio) [BDRip-720p-8bit]')

In [9]:
# series_2 = Series()
# series_2.collect_files('sample dir 2')
# series_2.remove_nonvideo_files()

In [10]:
playlist1 = SeriesPlaylistRandomizer()
playlist1.add_series('Cowboy Bebop',series_1)
xml = playlist1.get_randomized_playlist()

AttributeError: 'str' object has no attribute 'get_edited_paths'

In [ ]:
print(xml)

In [ ]:
# playlist2 = SeriesPlaylistRandomizer()
# playlist2.add_series('series 2',series_2)
# playlist2.get